# Lecture 6 : Real World

- Building *IO* system from scratch
- Introduce [IO](http://hackage.haskell.org/package/base/docs/Prelude.html#t:IO) monad
- **do** notation
    - Syntax sugar
    - [XApplicativeDo](https://downloads.haskell.org/~ghc/latest/docs/html/users_guide/glasgow_exts.html#applicative-do-notation)
    - [XRebindableSyntax](https://downloads.haskell.org/~ghc/latest/docs/html/users_guide/glasgow_exts.html#rebindable-syntax-and-the-implicit-prelude-import)
- Lazy I/O
- FFI
- Mutable data: [IORef](https://hackage.haskell.org/package/base/docs/Data-IORef.html#t:IORef) and [IOArray](http://hackage.haskell.org/package/array/docs/Data-Array-IO.html#t:IOArray)
- Exceptions ([catch](http://hackage.haskell.org/package/base/docs/Control-Exception-Base.html#v:catch), [throwIO](http://hackage.haskell.org/package/base/docs/Control-Exception-Base.html#v:throwIO), custom exceptions, [bracket](http://hackage.haskell.org/package/base/docs/Control-Exception.html#v:bracket), etc.)
- [unsafePerformIO](http://hackage.haskell.org/package/base/docs/System-IO-Unsafe.html#v:unsafePerformIO)
- Efficient String representations: [Text](http://hackage.haskell.org/package/text/docs/Data-Text.html#t:Text), [ByteString](http://hackage.haskell.org/package/bytestring/docs/Data-ByteString.html#t:ByteString)

## Building IO system from scratch

Хотим научиться работать с внешним миром.

Если мы хотим считывать что-то из какого-либо ввода, то у нас ничего не получится, потому что хаскель чистый и на одном входе нам должны возвращаться одни и те же значения:

### Начнем с простого getChar

```
getchar :: Char
get2chars = [getchar, getchar]
```
Problememes?
Haskell is pure!
```
let f a = a + 1
f 1 = 2
f 1 = 2
```

1. Because the Haskell compiler treats all functions as pure, it can avoid «excessive» calls to `getchar` and use one returned value twice.
 
2. Even if it does make two calls, there is no way to determine which call should be performed first. Do you want to return the two chars in the order in which they were read, or in the opposite order? Nothing in the definition of `get2chars` answers this question

Даже если мы как-то сделаем функцию getchar работающей, то хаскель ленивый. И никто не гарантирует, что она будет вызвана реально два раза, а не вернет значение первого вызова. Так же не гарантируется порядок этих вызовов.

Попробуем пофиксить:

```
getchar   :: Int -> Char
get2chars :: Int -> String
 
get2chars _ = [getchar 1, getchar 2]
getchar :: Int -> (Char, Int)

get2chars i = [a,b] where (a,i1) = getchar i
                          (b,i2) = getchar i1

-- Problems?

get4chars = [get2chars 1, get2chars 2]
```
order of 'get2chars' calls isn't defined

теперь 'get2chars' не понятно в каком порядке вызовиться

```
get2chars :: Int -> (String, Int)
get2chars i0 = ([a,b], i2)  where (a,i1) = getchar i0
                                  (b,i2) = getchar i1

get4chars :: Int -> String
get4chars i0 = (a++b) where (a,i1) = get2chars i0
                            (b,i2) = get2chars i
```

Believe it or not, but we've just constructed the whole "monadic" Haskell I/O system.

Хоба, по сути сделали реализацию IO в хаскеле. Однако вместо инте передаем более сложные вещи.

Можем апроксимировать подход так(до сих пор играем)

 ```
type IO a  =  RealWorld -> (a, RealWorld)

main :: RealWorld -> ((), RealWorld)
main :: IO ()

getChar :: RealWorld -> (Char, RealWorld)
 
main :: RealWorld -> ((), RealWorld)
main world0 = 
    let (a, world1) = getChar world0
        (b, world2) = getChar world1
    in ((), world2)
```


## Introduce IO monad

Теперь посмотрим на реальную IO monad

```
newtype IO a = IO {unIO :: State# RealWorld -> (State# RealWorld, a)}
Real implementation of IO type
{-# LANGUAGE MagicHash #-}  -- allows using # in names

data Mystery# a = Magic# a deriving (Show)
ghci> Magic# 3
Magic# 3
ghci> :t Magic# 3
Magic# 3 :: Num a => Mystery# a
```

Решетка в названии типа обозначает, что это что-то низкоуровневое, реализованное где-то в потрохах компилятора. Их используют только в лоулевел вещах.

State# а - будем считать просто оберткой, у него нет конструктора
RealWorld - токен для задания последовательных вычислений

Заворачиваем в монаду:

```
data State# s
data RealWorld

instance Monad IO where
  IO m >>= k = IO $ \ s -> 
     case m s of 
       (new_s, a) -> unIO (k a) new_s
  
  return x = IO (\s -> (s, x))
```


## **do** notation

### Syntax sugar

### >>

это синтаксический сахар над операцией >>

```
(>>) :: IO a -> IO b -> IO b
(action1 >> action2) world0 =
   let (_, world1) = action1 world0
       (b, world2) = action2 world1
   in (b, world2)
COPY
putStrLn :: String -> IO ()

main = do putStrLn "Hello!"
main = putStrLn "Hello!"
main = do putStrLn "What is your name?"
          putStrLn "How old are you?"
          putStrLn "Nice day!"

main = putStrLn "What is your name?" >>
       putStrLn "How old are you?"   >>
       putStrLn "Nice day!"
```
Применяя >> к двум IO мы последовательно изменяем RealWorld и возвращаем только результат выполнения функции в последней монаде. putStrLn - выводит строку на консоль. Соединив их с помощью >> мы последовательно выполняем запись в консоль. Это можно упростить с помощью do нотации, которая разворачивается в цепочку из монад и >>


Логично что тогда можем исполнять пос-ть действий, подумаем как это сделать.

```
ioActions :: [IO ()]
ioActions = [ print "Hello!"
            , putStr "just kidding"
            , getChar >> return ()
            ]

-- ugly

main = do head ioActions
          ioActions !! 1
          last ioActions

-- nice

sequence_ :: [IO a] -> IO ()

main = sequence_ ioActions

sequence_ :: [IO a] -> IO ()
sequence_ l =
sequence_ :: [IO a] -> IO ()
sequence_ []     = return ()
sequence_ (x:xs) = do x
                      sequence_ xs
```

Функция sequence_ принимает список монад и соединяет с помощью >> игнорируя результаты. Обычно функции, у которых _ на конце игнорируют возвращаемое значение. Есть функция sequence которая делает то же самое но возвращает результат последней монады



### <- 

```
(>>=) :: IO a -> (a -> RealWorld -> (b, RealWorld)) -> IO b

(action1 >>= action2) world0 =
   let (a, world1) = action1 world0
       (b, world2) = action2 a world1
   in (b, world2)


main = do s <- getLine
          putStrLn s

main = getLine >>= \s ->
       putStrLn s

main = getLine >>= putStrLn
```

Здесь показано как bind использовать в do нотации. Последние 3 блока являются идентичными


### GHCi

GHCi = infinite do-block inside IO

```
ghci> s <- getLine
Hello, world
ghci> putStrLn $ s ++ "!"
Hello, world!
```


### Еще дешугарим

```
main = do putStr "What is your name?"
          a <- readLn
          putStr "How old are you?"
          b <- readLn
          print (a,b)


main =    putStr "What is your name?" >>
          readLn >>= \a -> 
          putStr "How old are you?" >>
          readLn >>= \b -> 
          print (a,b)

```



### return!


```
return :: a -> IO a
return a world0 = (a, world0)
```

Первый пример - чтение с консоли строки и выведение ее в ревреснутом виде. return заворачивает значение в монаду, не изменяющую состояние. 
```
getReversedLine :: IO String
getReversedLine = do
    s <- getLine
    return $ reverse s

main :: IO ()
main = do
    rs <- getReversedLine
    putStrLn rs
```

Второй пример показывает, что return в ФП и в императивных языках - разные вещи. После выполнения первой ветки if будет выведено “a is positive”, так как return  не обрывает вычисления, а просто возвращает монаду, встраивающуюся в цепочку наших вычислений.

```
main = do a <- readLn
          if a >= 0 then 
              return ()
          else do
              putStrLn "a is negative"

          putStrLn "a is positive"  -- is this executed?

```

## **let** inside **do**

Можем использовать let в do

```
main :: IO ()
main = do
    s <- getLine
    let rs = reverse s
    putStrLn $ "Reversed input : " ++ rs
```
Раскрываеться в
```
main :: IO ()
main =     getLine >>= \s -> 
           let rs = reverse s in
           putStrLn $ "Reversed input : " ++ rs
```
Ошибочки
```
let s = getLine  -- !!! Doesn't read from console to `s`
rs <- reverse s  -- !!! `reverse s` is not a monadic action inside IO
```
Let используется в do нотации для хранения результатов немонадических вычислений

### **do** без монад

можем использовать без монад.
к примеру:
```
pythagoras :: Int -> Int -> Int
pythagoras x y = do
    let x2 = x ^ 2
    let y2 = y ^ 2
    x2 + y2


ghci> pythagoras 3 4 
25
```
И с функциями можем

```
foo = do
    a <- (+1)
    return (a * 2)
```



## XApplicativeDo

do нотация для аппликатива

## XRebindableSyntax

позваляет ребиндить нэймспэйс прелюда


## Lazy I/O

Возникают проблемы с ленивостью

```
main = do
  fileContent <- readFile "foo.txt"
  writeFile "bar.txt" ('a':fileContent)
  readFile  "bar.txt" >>= putStrLn


ghci> :run main
afoo
bar
```

\* что то поменли в коде, запустим еще раз\*

```

main = do
  fileContent <- readFile "foo.txt"
  writeFile "foo.txt" ('a':fileContent)
  readFile  "foo.txt" >>= putStrLn

ghci> :run main
*** Exception: foo.txt: openFile: resource busy (file is locked)
```

Так как хаскель ленивый, то во втором примере первый readFile не будет выполнен сразу. Он будет выполнен, когда это понадобится функции writeFile, которая к этому моменту уже открыла файл на запись. Соответственно ошибка появляется из-за того, что мы пытаемся прочесть файл, открытый на запись.

#### Решение: используйте современные библиотеки и почаще смотрите в документацию

## FFI

В хаскеле можно вызывать код, написанный на других языках, с помощью FFI:

### C
```
/* clang -c simple.c -o simple.o */

int example(int a, int b)
{
  return a + b;
}
```

### Haskell

```
-- ghc simple.o simple_ffi.hs -o simple_ffi

{-# LANGUAGE ForeignFunctionInterface #-}

import Foreign.C.Types

foreign import ccall safe "example" 
    example :: CInt -> CInt -> CInt

main = print (example 42 27)
```

## Mutable data: IORef and IOArray

Хотим изменяемые переменные... 

ого, проблемы

```
main = do let a0 = readVariable  varA
          let _  = writeVariable varA 1
          let a1 = readVariable  varA
          print (a0, a1)
```

У нас как всегда ничего не получится: ghci увидит, что результат writeVariable нигде не используется и проигнорирует вызов этой функции. Придется лепить монаду:

Используем IORef

```
import Data.IORef (newIORef, readIORef, writeIORef)

foo :: IO ()
foo = do 
    varA <- newIORef 0 
    a0   <- readIORef varA
    writeIORef varA 1
    a1   <- readIORef varA
    print (a0, a1)
```

Использовать такие переменные можно только внутри IO монады.

Так же есть изменяемые массивы:

```
import Data.Array.IO (IOArray, newArray, readArray, writeArray)

bar :: IO ()
bar = do 
    arr <- newArray (1,10) 37 :: IO (IOArray Int Int)
    a   <- readArray arr 1
    writeArray arr 1 64
    b   <- readArray arr 1
    print (a, b)
ghci> bar
(37,64)
```

Первый тип в IOArray - это тип индекса, второй - тип хранимого значения

Они не очень быстрые, лучше использовать Vector



## Exceptions (catch, throwIO, custom exceptions, bracket, etc.)

Ого, даже тут исключения

### throw

```
throwIO :: Exception e => e -> IO a
import           Control.Exception (ArithException (..), catch, throwIO)
import           Control.Monad     (when)

readAndDivide :: IO Int
readAndDivide = do
    x <- readLn
    y <- readLn
    when (y == 0) $ throwIO DivideByZero
    return $ x `div` y
ghci> readAndDivide 
7
3
2
ghci> readAndDivide 
3
0
*** Exception: divide by zero
```
Исключением может быть любой тип при определении нужного instance. Для deriving Exception к исключению будет просто применен show. throwIO заворачивает в монаду IO этот тип

1. IO can throw ANY exception.
2. Not represented in type system.

### catch

Ого, а теперь ловим

```
catch :: Exception e => IO a -> (e -> IO a) -> IO a
safeReadAndDivide :: IO Int
safeReadAndDivide = readAndDivide `catch` \DivideByZero -> return (-1)
ghci> safeReadAndDivide 
7
3
2
ghci> safeReadAndDivide 
3
0
-1
```

IO может кинуть любое исключение, но один catch может поймать исключение только одного типа. 

### свои ошибки
```
{-# LANGUAGE DeriveAnyClass     #-}
{-# LANGUAGE DeriveDataTypeable #-}

import           Control.Exception (Exception)
import           Data.Typeable     (Typeable)

data MyException = DummyException
    deriving (Show, Typeable, Exception)
ghci> throwIO DummyException 
*** Exception: DummyException

ghci> :{
ghci| throwIO DummyException `catch` \DummyException ->
ghci|     putStrLn "Dummy exception is thrown"
ghci| :}
Dummy exception is thrown
```

### try

```
try     :: Exception e => IO a -> IO (Either e a)
tryJust :: Exception e => (e -> Maybe b) -> IO a -> IO (Either b a)

finally :: IO a	 -- computation to run first
        -> IO b	 -- computation to run afterward (even if an exception was raised)
        -> IO a

-- | Like 'finally', but only performs the final action 
-- if there was an exception raised by the computation.
onException :: IO a -> IO b -> IO a
```
try - вернет Left с ошибкой или Right с вычисленным значеним
tryJust - то же самое, но с хендлером

### как должно быть

Берет ресурс, что-то с ним делает, отпускает ресурс и возвращает то, что с ним сделалось.

Idiomatic RAII implementation

```
bracket :: IO a         -- ^ computation to run first (\"acquire resource\")
        -> (a -> IO b)  -- ^ computation to run last (\"release resource\")
        -> (a -> IO c)  -- ^ computation to run in-between
        -> IO c         -- returns the value from the in-between computation
```



## Reminding of pattern guards

хоба, не ожидали?

а <- есть и в гвардах. Выступает как паттернматчинг

```
lookup :: FiniteMap -> Int -> Maybe Int

addLookup :: FiniteMap -> Int -> Int -> Maybe Int
addLookup env var1 var2
 | Just val1 <- lookup env var1
 , Just val2 <- lookup env var2
 = val1 + val2
{-...other equations...-}


strangeOperation :: [Int] -> Ordering
strangeOperation xs 
   | 7  <- sum xs
   , n  <- length xs
   , n  >= 5
   , n  <= 20 
   = EQ
   | 1  <- sum xs
   , 18 <- length xs
   , r  <- nub xs `compare` [1,2,3]
   , r /= EQ
   = r
   | otherwise
   = [3,1,2] `compare` xs

main = print $ strangeOperation ([5,7..21] ++ [20,19..4])

```

## unsafePerformIO

### Welcome to Dark side of IO

позваляет "подавлять" IO

```
import System.IO.Unsafe

foo :: ()
foo = unsafePerformIO $ putStrLn "foo"

bar :: String
bar = unsafePerformIO $ do
          putStrLn "bar"
          return "baz"

main = do let f = foo
          putStrLn bar
```

т.к. foo - чистая, а Хаскель ленивый, то в мэйне foo не будет вычесленна =>
вывод будет bar baz

### делаем грязь(unsafe)

#### Изначально

```
helper i = print i >> return i

main = do
    one <- helper 1
    two <- helper 2
    print $ one + two
```
#### step 1

т.к. считаем, что help чистая, то можем получить разный порядок в выводе. Как 123
321
...

```
import System.IO.Unsafe

helper i = unsafePerformIO $ print i >> return i

main = do
    let one = helper 1
    let two = helper 2
    print $ one + two
```
### step 1.2

Пытаемся исправить, теперь  two - форситься на эвал


```
import System.IO.Unsafe

helper i = print i >> return i

main = do
    one <- helper 1
    let two = unsafePerformIO $ helper 2
    print $ one + two
```

казалось бы

1. Run helper
2. Create a thunk to run helper
3. Evaluate one + two, forcing the helper 2 thunk to be evaluated in the process.
4. Print the result of one + two, a.k.a. 3.

получим 123

однако тоже возможно
1. Create and evaluate the helper 2 thunk.
2. Run helper
3. Print the result of one + two

-> 213

### ignore

как же работает unsafe?

через ignore

```
newtype IO a = IO {unIO :: State# RealWorld -> (State# RealWorld, a)}

instance Monad IO where
  IO m >>= IO k = IO $ \ s -> 
     case m s of 
       (new_s, a) -> unIO (k a) new_s
  
  return x = IO (\s -> (s, x))
```
можем избегать ворнингов за неиспользованные индефикаторы начиная их с "_"

```
type S# = State# RealWorld  -- let's use this short alias

print 1 :: S# -> (S#, ())

print 1 >> print 2 = 
    \s0 -> case print 1 s0 of
               (s1, _ignored) -> print 2 s1

unsafePerformIO (IO f) =
    case f fakeStateToken of
        (_ignoredStateToken, result) -> result
```

по факту unsafe использует фэйковый токен, полученный "магией"




### Desugaring IO

сахар

```
import System.IO.Unsafe

helper i = print i >> return i

main = do
    one <- helper 1
    let two = unsafePerformIO $ helper 2
    print $ one + two
```

без сахара

```
main s0 =
    case helper 1 s0 of
        (s1, one) ->
            case helper 2 fakeStateToken of
                (_ignored, two) ->
                    print (one + two) s1
```

### в результате

мораль не используйте.
Но можете, если уверенны, что как то посл-ть сохраниться

1. Whenever possible, avoid using unsafe functions.
 
2. If you aren't in the IO monad at all, or it's acceptable if the action is performed before other IO actions, use unsafePerformIO.



### Real life

Дебажим

```
import Debug.Trace

trace     :: String -> a -> a
traceShow :: Show a => a -> b -> b
traceM    :: Applicative f => String -> f ()
Debugging pure functions
trace :: String -> a -> a
trace string expr = unsafePerformIO $ do
    traceIO string  -- slightly clever version of `putStrLn`
    return expr
fib :: Int -> Int
fib 0 = 0
fib 1 = 1
fib n = trace ("n: " ++ show n) $ fib (n - 1) + fib (n - 2)
ghci> putStrLn $ "fib 4:\n" ++ show (fib 4)
fib 4:
n: 4
n: 3
n: 2
n: 2
3
```

## Efficient String representations: Text, ByteString

### String

Изначально String не очень то эффективен. На каждый char по 4 байта

### Пытаемс в другие представления

для начала сделаем возможность других представлений

```

{-# LANGUAGE OverloadedStrings #-}

class IsString a where
    fromString :: String -> a

ghci> :t "foo"
"foo" :: [Char]

ghci> :set -XOverloadedStrings

ghci> :t "foo"
"foo" :: IsString a => a
```

### Text

```
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.Text as T

-- From pack
myTStr1 :: T.Text
myTStr1 = T.pack ("foo" :: String)

-- From overloaded string literal.
myTStr2 :: T.Text
myTStr2 = "bar"
```

### ByteString

```
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.ByteString       as S
import qualified Data.ByteString.Char8 as S8

-- From pack
bstr1 :: S.ByteString
bstr1 = S.pack ("foo" :: String)

-- From overloaded string literal.
bstr2 :: S8.ByteString
bstr2 = "bar"
```

### ByteString and unsafePerformIO

под капотом ByteString на самом деле сишный указатель. Но мы хотим работать с ByteString и в чистом коде => unsafe

Text аналогично, только указатель не на Word8

```
data ByteString = PS (ForeignPtr Word8) Int Int

unsafeCreate :: Int -> (Ptr Word8 -> IO ()) -> ByteString
unsafeCreate l f = unsafePerformIO (create l f)

create :: Int -> (Ptr Word8 -> IO ()) -> IO ByteString
create l f = do
    fp <- mallocByteString l
    withForeignPtr fp $ \p -> f p
    return $ PS fp 0 l
    
-- | /O(1)/ 'splitAt' @n xs@ is equivalent to @('take' n xs, 'drop' n xs)@.
splitAt :: Int -> ByteString -> (ByteString, ByteString)

-- | /O(1)/ Extract the last element of a ByteString.
last :: ByteString -> Word8
last ps@(PS x s l)
    | null ps   = errorEmptyList "last"
    | otherwise = unsafePerformIO $
                    withForeignPtr x $ \p -> peekByteOff p (s+l-1)
```

Хоть и есть unsafe, но у нас прямые руки

### Что и зачем

1. Binary:
    - Packed:
        - Lazy: Data.ByteString.Lazy
        - Strict: Data.ByteString

2. Text:
    1. ASCII or 8-bit:
        - Packed and lazy: Data.ByteString.Lazy.Char8
        - Packed and strict: 
            - Data.ByteString.Char8, 
            - Data.CompactString.ASCII
            - Data.CompactString with Latin1

     2. Unicode:
        - UTF-32: 
            - Unpacked and lazy: [Char]
        - UTF-16:
            - Packed and lazy: Data.Text.Lazy
            - Packed and strict: Data.Text or Data.CompactString.UTF16
        - UTF-8:
            - Unpacked and lazy: Codec.Binary.UTF8.Generic contains generic operations that can be used   to process [Word8].
            - Packed and lazy: Data.ByteString.Lazy.UTF8
            - Packed and strict: Data.CompactString.UTF8 or Data.ByteString.UTF8


